# Chapter 9 - Up and Running with Tensorflow

## Import Libraries

In [1]:
import tensorflow as tf
import numpy as np

from sklearn.datasets import fetch_california_housing
from sklearn.preprocessing import StandardScaler
from datetime import datetime

/Applications/anaconda3/lib/python3.6/site-packages/h5py/__init__.py:34: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters


## Functions

In [2]:
def reset_graph(seed=42):
    tf.reset_default_graph()
    tf.set_random_seed(seed)
    np.random.seed(seed)

## Creating Your First Graph and Running It in a Session

In [3]:
reset_graph()

x = tf.Variable(3, name="x")
y = tf.Variable(4, name="y")
f = x*x*y + y + 2

In [4]:
sess = tf.Session()
sess.run(x.initializer)
sess.run(y.initializer)
result = sess.run(f)
print(result)
sess.close()

42


### Running Session in a Block

In [5]:
with tf.Session() as sess:
    x.initializer.run()
    y.initializer.run()
    result2 = f.eval()
    print(result2)

42


In [6]:
sess.close()

### Running Session w/ Global Initializer

In [7]:
init = tf.global_variables_initializer() # prepare an init node

In [8]:
with tf.Session() as sess:
    init.run() # actually initialize all the variables
    result = f.eval()

### Interactive Session (Jupyter Notebook/Python Shell)

In [9]:
sess = tf.InteractiveSession()
init.run()
result = f.eval()
print(result)
sess.close()

42


## Managing Graphs

In [10]:
reset_graph()

x1 = tf.Variable(1)
x1.graph is tf.get_default_graph()

True

In [11]:
graph = tf.Graph()

In [12]:
with graph.as_default():
    x2 = tf.Variable(2)

In [13]:
x2.graph is graph

True

In [14]:
x2.graph is tf.get_default_graph()

False

## Lifecycle of a Node Value

In [15]:
w = tf.constant(3)
x = w + 2
y = x + 5
z = x * 3

**The code below will evaluate x twice. A variable starts its life when an initializer is run, and it ends when the session is closed.**

In [16]:
with tf.Session() as sess:
    print(y.eval()) # 10
    print(z.eval()) # 15

10
15


**If you want to evaluate y and z efficently, without evaluating w and x twice, you must ask Tensorflow to evaluate both y and z in just one graph run.**

In [17]:
with tf.Session() as sess:
    y_val, z_val = sess.run([y, z])
    print(y_val)
    print(z_val)

10
15


## Linear Regression with Tensorflow

In [18]:
reset_graph()

housing = fetch_california_housing()
m, n = housing.data.shape
housing_data_plus_bias = np.c_[np.ones((m, 1)), housing.data]

In [19]:
housing.data.shape

(20640, 8)

In [20]:
housing_data_plus_bias.shape

(20640, 9)

In [21]:
X = tf.constant(housing_data_plus_bias, dtype=tf.float32, name="X")
y = tf.constant(housing.target.reshape(-1, 1), dtype=tf.float32, name="y")
XT = tf.transpose(X)
theta = tf.matmul(tf.matmul(tf.matrix_inverse(tf.matmul(XT, X)), XT), y)

In [22]:
with tf.Session() as sess:
    theta_value = theta.eval()

## Implementing Gradient Descent

In [23]:
scaler = StandardScaler()
scaled_housing_data_plus_bias = scaler.fit_transform(housing_data_plus_bias)

### Manually Computing the Gradients

In [24]:
reset_graph()

n_epochs = 1000
learning_rate = 0.01

In [25]:
X = tf.constant(scaled_housing_data_plus_bias, dtype=tf.float32, name="X")
y = tf.constant(housing.target.reshape(-1, 1), dtype=tf.float32, name="y")
theta = tf.Variable(tf.random_uniform([n + 1, 1], -1.0, 1.0), name="theta")
y_pred = tf.matmul(X, theta, name="predictions")
error = y_pred - y
mse = tf.reduce_mean(tf.square(error), name="mse")
gradients = 2/m * tf.matmul(tf.transpose(X), error)
training_op = tf.assign(theta, theta - learning_rate * gradients)

In [26]:
init = tf.global_variables_initializer()

In [27]:
with tf.Session() as sess:
    sess.run(init)
    
    for epoch in range(n_epochs):
        
        if epoch % 100 == 0:
            print("Epoch", epoch, "MSE =", mse.eval())
        sess.run(training_op)
        
    best_theta = theta.eval()

Epoch 0 MSE = 7.589867
Epoch 100 MSE = 4.8741293
Epoch 200 MSE = 4.8182063
Epoch 300 MSE = 4.812053
Epoch 400 MSE = 4.809471
Epoch 500 MSE = 4.8077292
Epoch 600 MSE = 4.806481
Epoch 700 MSE = 4.805583
Epoch 800 MSE = 4.8049335
Epoch 900 MSE = 4.8044662


### Using autodiff

In [28]:
reset_graph()

def my_func(a, b):
    z = 0
    for i in range(100):
        z = a * np.cos(z + i) + z * np.sin(b - i)
    return z

In [29]:
gradients = tf.gradients(mse, [theta])[0]

### Using an Optimizer

In [30]:
# optimizer = tf.train.GradientDescentOptimizer(learning_rate=learning_rate)
# training_op = optimizer.minimize(mse)

In [31]:
# optimizer = tf.train.MomentumOptimizer(learning_rate=learning_rate,
#                                       momentum=0.9)

## Feeding Data to the Training Algorithm

In [32]:
reset_graph()

A = tf.placeholder(tf.float32, shape=(None, 3))
B = A + 5

with tf.Session() as sess:
    B_val_1 = B.eval(feed_dict={A: [[1, 2, 3]]})
    B_val_2 = B.eval(feed_dict={A: [[4, 5, 6], [7, 8, 9]]})
    
print(B_val_1)

[[6. 7. 8.]]


In [33]:
print(B_val_2)

[[ 9. 10. 11.]
 [12. 13. 14.]]


### Mini-batch Gradient Descent

In [34]:
n_epochs = 1000
learning_rate = 0.01

In [35]:
reset_graph()

X = tf.placeholder(tf.float32, shape=(None, n + 1), name="X")
y = tf.placeholder(tf.float32, shape=(None, 1), name="y")

In [36]:
theta = tf.Variable(tf.random_uniform([n + 1, 1], -1.0, 1.0, seed=42), name="theta")
y_pred = tf.matmul(X, theta, name="predictions")
error = y_pred - y
mse = tf.reduce_mean(tf.square(error), name="mse")
optimizer = tf.train.GradientDescentOptimizer(learning_rate=learning_rate)
training_op = optimizer.minimize(mse)

init = tf.global_variables_initializer()

In [37]:
n_epochs = 10

In [38]:
batch_size = 100
n_batches = int(np.ceil(m / batch_size))

In [39]:
def fetch_batch(epoch, batch_index, batch_size):
    np.random.seed(epoch * n_batches + batch_index)  
    indices = np.random.randint(m, size=batch_size) 
    X_batch = scaled_housing_data_plus_bias[indices]
    y_batch = housing.target.reshape(-1, 1)[indices] 
    return X_batch, y_batch

with tf.Session() as sess:
    sess.run(init)

    for epoch in range(n_epochs):
        for batch_index in range(n_batches):
            X_batch, y_batch = fetch_batch(epoch, batch_index, batch_size)
            sess.run(training_op, feed_dict={X: X_batch, y: y_batch})

    best_theta = theta.eval()

In [40]:
best_theta

array([[-0.1673944 ],
       [ 0.86515963],
       [ 0.11857292],
       [-0.29859844],
       [ 0.39628983],
       [-0.00649126],
       [ 0.00409026],
       [-0.80076975],
       [-0.7800274 ]], dtype=float32)

## Saving and Restoring Models

In [41]:
reset_graph()

n_epochs = 1000                                                                       
learning_rate = 0.01                                                                  

X = tf.constant(scaled_housing_data_plus_bias, dtype=tf.float32, name="X")            
y = tf.constant(housing.target.reshape(-1, 1), dtype=tf.float32, name="y")            
theta = tf.Variable(tf.random_uniform([n + 1, 1], -1.0, 1.0, seed=42), name="theta")
y_pred = tf.matmul(X, theta, name="predictions")                                      
error = y_pred - y                                                                    
mse = tf.reduce_mean(tf.square(error), name="mse")                                    
optimizer = tf.train.GradientDescentOptimizer(learning_rate=learning_rate)            
training_op = optimizer.minimize(mse)                                                 

init = tf.global_variables_initializer()
saver = tf.train.Saver()

with tf.Session() as sess:
    sess.run(init)

    for epoch in range(n_epochs):
        if epoch % 100 == 0:
            print("Epoch", epoch, "MSE =", mse.eval())                                
            save_path = saver.save(sess, "/tmp/my_model.ckpt")
        sess.run(training_op)
    
    best_theta = theta.eval()
    save_path = saver.save(sess, "/tmp/my_model_final.ckpt")

Epoch 0 MSE = 8.440994
Epoch 100 MSE = 4.9055037
Epoch 200 MSE = 4.8440895
Epoch 300 MSE = 4.8344755
Epoch 400 MSE = 4.827744
Epoch 500 MSE = 4.8225675
Epoch 600 MSE = 4.818563
Epoch 700 MSE = 4.8154435
Epoch 800 MSE = 4.813001
Epoch 900 MSE = 4.81108


In [42]:
best_theta

array([[-0.1673944 ],
       [ 0.8874027 ],
       [ 0.14401652],
       [-0.34770885],
       [ 0.3617837 ],
       [ 0.0039381 ],
       [-0.04269556],
       [-0.661453  ],
       [-0.63752794]], dtype=float32)

In [43]:
with tf.Session() as sess:
    saver.restore(sess, "/tmp/my_model_final.ckpt")
    best_theta_restored = theta.eval()

INFO:tensorflow:Restoring parameters from /tmp/my_model_final.ckpt


In [44]:
np.allclose(best_theta, best_theta_restored)

True

In [45]:
saver = tf.train.Saver({"weights": theta})

## Visualizing the Graph and Training Curves Using TensorBoard

In [46]:
reset_graph()

now = datetime.utcnow().strftime("%Y%m%d%H%M%S")
root_logdir = "tf_logs"
logdir = "{}/run-{}/".format(root_logdir, now)

In [47]:
n_epochs = 1000
learning_rate = 0.01

X = tf.placeholder(tf.float32, shape=(None, n + 1), name="X")
y = tf.placeholder(tf.float32, shape=(None, 1), name="y")
theta = tf.Variable(tf.random_uniform([n + 1, 1], -1.0, 1.0, seed=42), name="theta")
y_pred = tf.matmul(X, theta, name="predictions")
error = y_pred - y
mse = tf.reduce_mean(tf.square(error), name="mse")
optimizer = tf.train.GradientDescentOptimizer(learning_rate=learning_rate)
training_op = optimizer.minimize(mse)

init = tf.global_variables_initializer()

In [48]:
mse_summary = tf.summary.scalar('MSE', mse)
file_writer = tf.summary.FileWriter(logdir, tf.get_default_graph())

In [49]:
n_epochs = 10
batch_size = 100
n_batches = int(np.ceil(m / batch_size))

In [50]:
with tf.Session() as sess:                                                       
    sess.run(init)                                                               

    for epoch in range(n_epochs):                                                 
        for batch_index in range(n_batches):
            X_batch, y_batch = fetch_batch(epoch, batch_index, batch_size)
            if batch_index % 10 == 0:
                summary_str = mse_summary.eval(feed_dict={X: X_batch, y: y_batch})
                step = epoch * n_batches + batch_index
                file_writer.add_summary(summary_str, step)
            sess.run(training_op, feed_dict={X: X_batch, y: y_batch})

    best_theta = theta.eval()                                                     

In [51]:
file_writer.close()

In [52]:
best_theta

array([[-0.1673944 ],
       [ 0.86515963],
       [ 0.11857292],
       [-0.29859844],
       [ 0.39628983],
       [-0.00649126],
       [ 0.00409026],
       [-0.80076975],
       [-0.7800274 ]], dtype=float32)

## Name Scopes

In [53]:
reset_graph()

now = datetime.utcnow().strftime("%Y%m%d%H%M%S")
root_logdir = "tf_logs"
logdir = "{}/run-{}/".format(root_logdir, now)

n_epochs = 1000
learning_rate = 0.01

X = tf.placeholder(tf.float32, shape=(None, n + 1), name="X")
y = tf.placeholder(tf.float32, shape=(None, 1), name="y")
theta = tf.Variable(tf.random_uniform([n + 1, 1], -1.0, 1.0, seed=42), name="theta")
y_pred = tf.matmul(X, theta, name="predictions")

In [54]:
with tf.name_scope("loss") as scope:
    error = y_pred - y
    mse = tf.reduce_mean(tf.square(error), name="mse")

In [55]:
optimizer = tf.train.GradientDescentOptimizer(learning_rate=learning_rate)
training_op = optimizer.minimize(mse)

init = tf.global_variables_initializer()

mse_summary = tf.summary.scalar('MSE', mse)
file_writer = tf.summary.FileWriter(logdir, tf.get_default_graph())

In [56]:
n_epochs = 10
batch_size = 100
n_batches = int(np.ceil(m / batch_size))

with tf.Session() as sess:
    sess.run(init)

    for epoch in range(n_epochs):
        for batch_index in range(n_batches):
            X_batch, y_batch = fetch_batch(epoch, batch_index, batch_size)
            if batch_index % 10 == 0:
                summary_str = mse_summary.eval(feed_dict={X: X_batch, y: y_batch})
                step = epoch * n_batches + batch_index
                file_writer.add_summary(summary_str, step)
            sess.run(training_op, feed_dict={X: X_batch, y: y_batch})

    best_theta = theta.eval()

file_writer.flush()
file_writer.close()
print("Best theta:")
print(best_theta)

Best theta:
[[-0.1673944 ]
 [ 0.86515963]
 [ 0.11857292]
 [-0.29859844]
 [ 0.39628983]
 [-0.00649126]
 [ 0.00409026]
 [-0.80076975]
 [-0.7800274 ]]
